###Ingestion del archivo "language_role.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

####Paso 1- Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
language_role_schema = StructType(fields=[
    StructField("roleId", IntegerType(), False),
    StructField("languageRole", StringType(), True)
])

In [0]:
language_role_df = spark.read \
                .schema(language_role_schema) \
                .option("multiLine", True) \
                .json(f"{bronze_folder_path}/{v_file_date}/language_role.json")

In [0]:
display(language_role_df)

####Paso 2 - Renombrar las columnas y añadir nuevas columnas
- "roleId" renombar a "role_id", "languageRole" a "language_role"
- Agregar las columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
language_role_with_columns_df = add_ingestion_dat(language_role_df) \
                            .withColumnRenamed("languageRole","language_role") \
                            .withColumn("ingestionDate", current_timestamp()) \
                            .withColumn("environment", lit(v_environment))  \
                            .withColumn("file_date",lit(v_file_date))
display(language_role_with_columns_df)

####Paso 3 - Escribir la salida en un formato "Parquet"


In [0]:
#language_role_with_columns_df.write.mode("overwrite").parquet(f"{silver_folder_path}/language_role")
language_role_with_columns_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.language_role")

In [0]:
#language_role_with_columns_df.write.mode("overwrite").parquet(f"{silver_folder_path}/language_role")

In [0]:
#display(spark.read.parquet("/mnt/mymoviehistory/silver/language_role"))

In [0]:
dbutils.notebook.exit("Success")